# Evaluating performance

## Assignments

As in previous checkpoints, please submit links to two Juypyter notebooks (one for each assignment below).

Please submit links to all your work below. This is not a graded checkpoint, but you should discuss your solutions with your mentor. Also, when you're done, compare your work to [these example solutions](https://github.com/Thinkful-Ed/machine-learning-regression-problems/blob/master/notebooks/5.solution_evaluating_goodness_of_fit.ipynb).



### 1. Weather model

For this assignment, you'll revisit the historical temperature dataset. To complete this assignment, submit a link a Jupyter notebook containing your solutions to the following tasks:

* First, load the dataset from the **weatherinszeged** table from Thinkful's database.
* Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the *apparenttemperature* and the *temperature*. As explanatory variables, use *humidity* and *windspeed*. Now, estimate your model using OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why? 
* Next, include the interaction of *humidity* and *windspeed* to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one? 
* Add *visibility* as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the *visibility* in terms of the improvement in the adjusted R-squared. Which one is more useful?
* Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.


###  2. House prices model

In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* Load the **houseprices** data from Thinkful's database.
* Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
* Do you think your model is satisfactory? If so, why?
* In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables. 
* For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
temp_hist = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


temp_hist.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [3]:
temp_data = pd.DataFrame()
temp_data['target'] = temp_hist['apparenttemperature'] - temp_hist['temperature']
temp_data['humidity'] = temp_hist['humidity']
temp_data['windspeed'] = temp_hist['windspeed']

In [4]:
# We create a LinearRegression model object
lrm = linear_model.LinearRegression()

# We then select data and target 
data = temp_data.iloc[:, 1:]
target = temp_data['target']

# fit method estimates the coefficients using OLS
lrm.fit(data, target)

# Next we take a look at the results
# We need to manually add a constant in statsmodels' sm
data = sm.add_constant(data)

results = sm.OLS(target, data).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Mon, 09 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:06:13   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-squared: 0.288
Adj. R-squared:	0.288

I do not believe these results are satisfactory, that is a relatively small amount of the target to explain.     

In [5]:
# First select data and target 
data2 = temp_data.iloc[:, 1:]
data2['humidity_windspeed'] = temp_hist['humidity'] * temp_hist['windspeed']
target = temp_data['target']

# We create a LinearRegression model object
lrm = linear_model.LinearRegression()

# We need to manually add a constant in statsmodels' sm
data2 = sm.add_constant(data2)

# fit method estimates the coefficients using OLS
lrm.fit(data2, target)

results = sm.OLS(target, data2).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Mon, 09 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:06:13   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0.033      2.511      0.012       0.018       0.149
humidity               0.1775      0.043      4.133      0.000       0.093       0.262
windspeed              0.0905      0.002     36.797      0.000       0.086       0.095
humidity_windspeed    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-squared:	0.341
Adj. R-squared:	0.341

This model does a better job, but not by much. 

In [6]:
# We create a LinearRegression model object
lrm = linear_model.LinearRegression()

# We then select data and target 
data3 = temp_data.iloc[:, 1:]
data3['visibility'] = temp_hist['visibility']
target = temp_data['target']

# fit method estimates the coefficients using OLS
lrm.fit(data3, target)

# Next we take a look at the results
# We need to manually add a constant in statsmodels' sm
data3 = sm.add_constant(data3)

results = sm.OLS(target, data3).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Mon, 09 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:06:13   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-squared:	0.304
Adj. R-squared:	0.303

This model did not preform as well in regards to R-squared and Adj. R-squared in relation to the model with the interaction term included in it. 

Based on the results from each summary, the lower the BIC and AIC the better the model. SO in this case I'd choose the second one. 

###  2. House prices model

In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* Load the **houseprices** data from Thinkful's database.
* Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
* Do you think your model is satisfactory? If so, why?
* In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables. 
* For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [7]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
homes_df = pd.read_sql_query('select * from houseprices',con=engine)

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
homes_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


In [8]:
# Preparing data for modeling about house prices 

# ojects holding columns
non_numeric_columns = homes_df.select_dtypes(['object']).columns
numeric_columns = homes_df.select_dtypes(['int64', 'float64']).columns

# dropping columns Missing data
homes_df = homes_df.drop(['poolqc', 'miscfeature', 'alley', 
                          'fence', 'fireplacequ', 'lotfrontage'], axis=1)

# Dropping missing observations
homes_df = homes_df.dropna(axis=0)

numeric_columns = numeric_columns.drop(['id'])

FILL_LIST = []
for cols in homes_df[:]:
    if cols in numeric_columns:
        FILL_LIST.append(cols)

In [9]:
from scipy.stats.mstats import winsorize

homes_win = homes_df.copy()

for col in FILL_LIST:
    homes_win[col] = winsorize(homes_win[col], (.05, .14))

In [10]:
from sklearn import preprocessing

def cat_converter(df):
    for cols in df:
        if cols in non_numeric_columns:
            
            # Create a label (category) encoder object
            le = preprocessing.LabelEncoder()
            
            # Create a label (category) encoder object
            le.fit(df[cols])
            
            # Apply the fitted encoder to the pandas column
            df[cols] = le.transform(df[cols]) 
    return df

cat_converter(homes_win)

,id,mssubclass,mszoning,lotarea,street,lotshape,landcontour,utilities,lotconfig,landslope,...,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,3,8450,1,3,3,0,4,0,...,0,0,0,0,0,2,2008,8,4,208500
1,2,20,3,9600,1,3,3,0,2,0,...,0,0,0,0,0,5,2007,8,4,181500
2,3,60,3,11250,1,0,3,0,4,0,...,0,0,0,0,0,9,2008,8,4,223500
3,4,70,3,9550,1,0,3,0,0,0,...,0,0,0,0,0,2,2006,8,0,140000
4,5,60,3,13518,1,0,3,0,2,0,...,0,0,0,0,0,10,2008,8,4,250000
5,6,50,3,13518,1,0,3,0,4,0,...,0,0,0,0,0,10,2009,8,4,143000
6,7,20,3,10084,1,3,3,0,4,0,...,0,0,0,0,0,8,2007,8,4,261500
7,8,60,3,10382,1,0,3,0,0,0,...,0,0,0,0,0,10,2009,8,4,200000
8,9,50,4,6120,1,3,3,0,4,0,...,0,0,0,0,0,4,2008,8,0,129900
9,10,90,3,7420,1,3,3,0,0,0,...,0,0,0,0,0,2,2008,8,4,118000


In [11]:
# selecting data and target
homes_mod1 = homes_win[['lotarea', 'bsmtfinsf1', 'grlivarea', 'saleprice']]

In [12]:
# We create a LinearRegression model object
lrm = linear_model.LinearRegression()

data = homes_mod1.iloc[:, :-1]
target = homes_mod1['saleprice']

# fit method estimates the coefficients using OLS
lrm.fit(data, target)

# We need to manually add a constant
# in statsmodels' sm
data = sm.add_constant(data)

results = sm.OLS(target, data).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.596
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     656.2
Date:                Mon, 09 Dec 2019   Prob (F-statistic):          5.64e-262
Time:                        21:06:20   Log-Likelihood:                -15812.
No. Observations:                1338   AIC:                         3.163e+04
Df Residuals:                    1334   BIC:                         3.165e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8865.0473   4024.634      2.203      0.028     969.747    1.68e+04
lotarea        1.9548      0.344      5.689      0.000       1.281       2.629
bsmtfinsf1    29.5989      2.487     11.902      0.000      24.720      34.478
grlivarea     92.6300      2.620     35.349      0.000      87.489      97.771
==============================================================================
Omnibus:                       28.273   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.336
Skew:                          -0.276   Prob(JB):                     3.50e-08
Kurtosis:                       3.558   Cond. No.                     4.49e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.49e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Just interpreting R, F, BIC, & AIC

The R-squared and Adj R-squared values are right around .595, which is okay but not great. 

The Prob of F-statistic seems really good as its really low and compared to our previous models the AIC and BIC returns seem better. 

I don't think the model is that great, explanatory power can be improved and a double check for multicolinearity should be done. 

In [13]:
homes_mod2 = homes_win[['neighborhood', 'bsmtfinsf1', 'heatingqc', 'grlivarea', 'saleprice']]

In [14]:
# We create a LinearRegression model object
lrm = linear_model.LinearRegression()

data = homes_mod2.iloc[:, :-1]
target = homes_mod2['saleprice']

# fit method estimates the coefficients using OLS
lrm.fit(data, target)

# We need to manually add a constant
# in statsmodels' sm
data = sm.add_constant(data)

results = sm.OLS(target, data).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.658
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     642.1
Date:                Mon, 09 Dec 2019   Prob (F-statistic):          6.07e-309
Time:                        21:06:20   Log-Likelihood:                -15700.
No. Observations:                1338   AIC:                         3.141e+04
Df Residuals:                    1333   BIC:                         3.144e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         4.116e+04   4013.035     10.256      0.000    3.33e+04     4.9e+04
neighborhood   458.3142    138.754      3.303      0.001     186.115     730.514
bsmtfinsf1      31.4326      2.274     13.825      0.000      26.972      35.893
heatingqc    -8186.5206    496.992    -16.472      0.000   -9161.492   -7211.549
grlivarea       87.2021      2.304     37.842      0.000      82.681      91.723
==============================================================================
Omnibus:                       30.546   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               34.408
Skew:                          -0.321   Prob(JB):                     3.38e-08
Kurtosis:                       3.454   Cond. No.                     7.71e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.71e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The second model with different features preformed better during Adj. R-squared, AIC, and BIC scores. So the second model is better, but there is still work to be done. 